#### General Steps to Follow

1. Importing Packages
1. Load Data and Build Generator & Discriminator Models
1. Training GAN
1. Generating Images
1. Saving the Generator Model

### --------------------------------------------------------------------------------------------------------------------------------------------------------

## 1) Importing Packages

#### Add the repository directory path to the Python path

In [1]:
import os
import sys

REPO_DIR_PATH = os.path.normpath(os.path.join(
    os.path.join(os.path.dirname(os.getcwd()))))

sys.path.append(REPO_DIR_PATH)

In [3]:
from src.train_model_cgan import ModelTrainerCGAN
import tensorflow as tf
import matplotlib.pyplot as plt

### --------------------------------------------------------------------------------------------------------------------------------------------------------

## 2) Load Data and Build Generator & Discriminator Models

In [4]:
trainer = ModelTrainerCGAN()

2024-08-04 16:05:20.231532: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-04 16:05:20.235976: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
real_images, real_labels = trainer.load_data()
real_images.shape, real_labels.shape

((60000, 28, 28, 1), (60000,))

In [7]:
EPOCHS = 100
BATCH_SIZE = 256
NOISE_DIM = 100
BUFFER_SIZE = 60000
NUM_CLASSES = 10

In [8]:
generator = trainer.build_generator(NUM_CLASSES, NOISE_DIM)
generator.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1, 100)    │      1,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 100)       │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 200)       │          0 │ input_layer[0][0… │
│ (Concatenate)       │                   │            │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 12544)     │  2,508,800 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 12544)     │     50,176 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 12544)     │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 7, 7, 256) │          0 │ leaky_re_lu[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 7, 7, 128) │    819,200 │ reshape[0][0]     │
│ (Conv2DTranspose)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 128) │        512 │ conv2d_transpose… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_1       │ (None, 7, 7, 128) │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, 14, 14,    │    204,800 │ leaky_re_lu_1[0]… │
│ (Conv2DTranspose)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 14, 14,    │        256 │ conv2d_transpose… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_2       │ (None, 14, 14,    │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_2  │ (None, 28, 28, 1) │      1,600 │ leaky_re_lu_2[0]… │
│ (Conv2DTranspose)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,586,344 (13.68 MB)

 Trainable params: 3,560,872 (13.58 MB)

 Non-trainable params: 25,472 (99.50 KB)

In [9]:
discriminator = trainer.build_discriminator(NUM_CLASSES)
discriminator.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 784)    │      7,840 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 784)       │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 28, 28, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 28, 28, 1) │          0 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 28, 28, 2) │          0 │ input_layer_2[0]… │
│ (Concatenate)       │                   │            │ reshape_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 14, 14,    │      3,264 │ concatenate_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_3       │ (None, 14, 14,    │          0 │ conv2d[0][0]      │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 14, 14,    │          0 │ leaky_re_lu_3[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 7, 7, 128) │    204,928 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_4       │ (None, 7, 7, 128) │          0 │ conv2d_1[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 7, 7, 128) │          0 │ leaky_re_lu_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 6272)      │          0 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │      6,273 │ flatten_2[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 222,305 (868.38 KB)

 Trainable params: 222,305 (868.38 KB)

 Non-trainable params: 0 (0.00 B)

### --------------------------------------------------------------------------------------------------------------------------------------------------------

## 3) Training GAN

In [ ]:
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices((real_images, real_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
trainer.train(train_dataset, NOISE_DIM, generator, discriminator, EPOCHS, BATCH_SIZE, NUM_CLASSES)

2024-08-04 16:08:36.798570: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2024-08-04 16:08:36.899885: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


### --------------------------------------------------------------------------------------------------------------------------------------------------------

## 4) Generating Images

In [2]:
def generate_images(generator_model, noise_dim, num_classes) -> plt.Figure:
    """
    Generate images using generator model.
    Args:
        generator_model: tensorflow.keras.models.Model, generator model
        noise_dim: int, dimension of the noise vector
        num_classes: int, number of classes for the labels
    Returns:
        fig: matplotlib.pyplot.Figure, figure containing the generated images
    """
    # Generate random noise and labels
    noise = tf.random.normal([16, noise_dim])
    labels = tf.random.uniform([16], minval=0, maxval=num_classes, dtype=tf.int32)

    # Generate images
    generated_images = generator_model([noise, labels], training=False)

    # Convert generated images from [-1, 1] to [0, 1]
    generated_images = (generated_images + 1) / 2.0

    # Plot images and their labels
    fig, axes = plt.subplots(4, 4, figsize=(10, 10))
    axes = axes.flatten()

    for img, label, ax in zip(generated_images, labels, axes):
        ax.imshow(img.numpy().squeeze(), cmap='gray')
        ax.set_title(f'Label: {int(label)}')
        ax.axis('off')

    plt.tight_layout()
    return fig

NameError: name 'plt' is not defined

In [ ]:
f = generate_images(generator, NOISE_DIM, NUM_CLASSES)

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, Conv2DTranspose, BatchNormalization, Reshape, LeakyReLU, Embedding, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
import numpy as np

2024-08-04 15:21:57.310637: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-04 15:21:58.001118: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
input_noise = Input(shape=(100,))
input_label = Input(shape=(1,))
embedding = Embedding(10, 100)
emb = embedding(input_label)
flat = Flatten()
model = Model(input_label, flat(emb))
X = np.array([3,4,5])
y = model(X)
y.shape

2024-08-04 15:21:58.986772: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-04 15:21:58.991905: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


TensorShape([3, 100])

In [7]:
    def build_generator( num_classes: int, noise_dim: int, input_noise_shape: tuple = (100,)) -> tf.keras.Model:
        """
        Build generator model
        The output shape of the generator model is (28, 28, 1)
        Args:
            input_shape: tuple, input shape of the generator model
        Returns:
            generator: tf.keras.Model, generator model
        """
        # Input layers for the noise and the label
        input_noise = Input(shape=input_noise_shape)
        input_label = Input(shape=(1,), dtype='int32')

        # Output from the embedding layer will be (None, 1, 100)
        embedding = Embedding(num_classes, noise_dim)(input_label)

        # Reshape the output to (None, 100)
        embedding = Flatten()(embedding)

        # Concatenate the noise and the label
        model_input = Concatenate()([input_noise, embedding])

        x = Dense(7*7*256, use_bias=False)(model_input)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)

        x = Reshape((7, 7, 256))(x)

        x = Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)

        x = Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)

        y = Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')(x)

        generator = Model([input_noise, input_label], y)

        return generator

In [8]:
generator = build_generator(10,100)
generator.compile(loss="binary_cross_entropy", optimizer = "Adam")
generator.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 100)    │      1,000 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 100)       │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 200)       │          0 │ input_layer_4[0]… │
│ (Concatenate)       │                   │            │ flatten_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 12544)     │  2,508,800 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 12544)     │     50,176 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 12544)     │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 7, 7, 256) │          0 │ leaky_re_lu[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 7, 7, 128) │    819,200 │ reshape[0][0]     │
│ (Conv2DTranspose)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 128) │        512 │ conv2d_transpose… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_1       │ (None, 7, 7, 128) │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, 14, 14,    │    204,800 │ leaky_re_lu_1[0]… │
│ (Conv2DTranspose)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 14, 14,    │        256 │ conv2d_transpose… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_2       │ (None, 14, 14,    │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_2  │ (None, 28, 28, 1) │      1,600 │ leaky_re_lu_2[0]… │
│ (Conv2DTranspose)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,586,344 (13.68 MB)

 Trainable params: 3,560,872 (13.58 MB)

 Non-trainable params: 25,472 (99.50 KB)

In [9]:
# noise = tf.random.normal([1,100])
# labels = np.random.randint(0, 10, 1).reshape(-1, 1)
# generator.predict([noise,labels])

In [11]:
# batch_size = 16
noise = tf.random.normal([1,100], dtype=tf.float32)
labels = tf.random.uniform([1, 1], minval=0, maxval=10, dtype=tf.int32)
generated_images = generator.predict([noise, labels])
generated_images.shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step


(1, 28, 28, 1)

In [60]:
generator.input

[<KerasTensor shape=(None, 100), dtype=float32, sparse=None, name=keras_tensor_100>,
 <KerasTensor shape=(None, 1), dtype=int32, sparse=None, name=keras_tensor_101>]

In [61]:
generator.output

<KerasTensor shape=(None, 28, 28, 1), dtype=float32, sparse=False, name=keras_tensor_115>

In [169]:
noise = tf.random.normal([1, 100], dtype=tf.float32)
labels = tf.random.uniform([2, 1], minval=0, maxval=10, dtype=tf.int32)
noise.shape

TensorShape([1, 100])

In [166]:
labels

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[2],
       [5]], dtype=int32)>

In [177]:
# Input layers for the noise and the label
input_noise = Input(shape=(100,))
input_label = Input(shape=(1,), dtype='int32')

# Output from the embedding layer will be (None, 1, 100)
embedding = Embedding(10, 100)(input_label)

# Reshape the output to (None, 100)
embedding = Flatten()(embedding)

# Concatenate the noise and the label
model_input = Concatenate()([input_noise, embedding])
m = Model([input_noise, input_label], model_input)

In [178]:
m.output

<KerasTensor shape=(None, 200), dtype=float32, sparse=False, name=keras_tensor_148>

In [27]:
input_image_shape = (28,28,1)
num_classes = 10
input_image = Input(shape=(input_image_shape))
input_label = Input(shape=(1,), dtype='int32')

# Output from the embedding layer will be (None, 1, 784)
embedding = Embedding(num_classes, np.prod(input_image_shape))(input_label)
m1 = Model(input_image, embedding)
# Reshape the output to (None, 28, 28, 1)
embedding = Flatten()(embedding)
m2 = Model(input_image, embedding)

# Reshape the image to (None, 28, 28, 1)
embedding = Reshape(input_image_shape)(embedding)

# Concatenate the image and the label
model_input = Concatenate(axis=-1)([input_image, embedding])
m3 = Model(input_image, embedding)
m4 = Model(input_image, model_input)

In [28]:
m1.output

<KerasTensor shape=(None, 1, 784), dtype=float32, sparse=False, name=keras_tensor_51>

In [29]:
m2.output

<KerasTensor shape=(None, 784), dtype=float32, sparse=False, name=keras_tensor_52>

In [30]:
m3.output

<KerasTensor shape=(None, 28, 28, 1), dtype=float32, sparse=False, name=keras_tensor_53>

In [31]:
m4.output

<KerasTensor shape=(None, 28, 28, 2), dtype=float32, sparse=False, name=keras_tensor_54>

In [32]:
def build_discriminator(num_classes: int, input_image_shape: tuple = (28,28,1)) -> tf.keras.Model:
    """
    Build discriminator model
    Args:
        num_classes: int, number of classes
    Returns:
        discriminator: tf.keras.Model, discriminator model
    """
    # Input layers for the image and the label
    input_image = Input(shape=input_image_shape)
    input_label = Input(shape=(1,), dtype='int32')

    # Output from the embedding layer will be (None, 1, 784)
    embedding = Embedding(num_classes, np.prod(input_image_shape))(input_label)

    # Reshape the output to (None, 784)
    embedding = Flatten()(embedding)

    # Reshape the image to (None, 28, 28, 1)
    embedding = Reshape(input_image_shape)(embedding)

    # Concatenate the image and the label
    # The shape of the model_input will be (None, 28, 28, 2)
    model_input = Concatenate()([input_image, embedding])

    x = Conv2D(64, (5, 5), strides=(2, 2), padding='same')(model_input)
    x = LeakyReLU()(x)
    x = Dropout(0.3)(x)

    x = Conv2D(128, (5, 5), strides=(2, 2), padding='same')(x)
    x = LeakyReLU()(x)
    x = Dropout(0.3)(x)

    x = Flatten()(x)
    y = Dense(1, activation='linear')(x)

    discriminator = Model([input_image, input_label], y)

    return discriminator

In [34]:
d = build_discriminator(10)
d.summary()

Model: "functional_43"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_19      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_9         │ (None, 1, 784)    │      7,840 │ input_layer_19[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_10          │ (None, 784)       │          0 │ embedding_9[0][0] │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_18      │ (None, 28, 28, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_6 (Reshape) │ (None, 28, 28, 1) │          0 │ flatten_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_7       │ (None, 28, 28, 2) │          0 │ input_layer_18[0… │
│ (Concatenate)       │                   │            │ reshape_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 14, 14,    │      3,264 │ concatenate_7[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_5       │ (None, 14, 14,    │          0 │ conv2d_2[0][0]    │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 14, 14,    │          0 │ leaky_re_lu_5[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 7, 7, 128) │    204,928 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_6       │ (None, 7, 7, 128) │          0 │ conv2d_3[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 7, 7, 128) │          0 │ leaky_re_lu_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_11          │ (None, 6272)      │          0 │ dropout_3[0][0]   │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │      6,273 │ flatten_11[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 222,305 (868.38 KB)

 Trainable params: 222,305 (868.38 KB)

 Non-trainable params: 0 (0.00 B)

In [43]:
image = tf.random.normal([1,28,28,1])
label = tf.random.uniform((1,1), minval=0, maxval=10)
d.predict([image,label])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


array([[0.13221194]], dtype=float32)

In [38]:
X_train = np.load("../data/external/mnist_x_train.npy")
y_train = np.load("../data/external/mnist_y_train.npy")
X_test = np.load("../data/external/mnist_x_test.npy")
y_test = np.load("../data/external/mnist_y_test.npy")

In [39]:
X_train.shape

(60000, 28, 28)

In [40]:
y_train.shape

(60000,)

In [46]:
p = d.predict([X_train, y_train])

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step


In [47]:
p.shape

(60000, 1)

In [49]:
def generate_images(generator_model: Model, noise_dim: int, num_classes: int) -> plt.Figure:
    """
    Generate images using generator model.
    Args:
        generator_model: tensorflow.keras.models.Model, generator model
        noise_dim: int, dimension of the noise vector
        num_classes: int, number of classes for the labels
    Returns:
        fig: matplotlib.pyplot.Figure, figure containing the generated images
    """
    # Generate random noise and labels
    noise = tf.random.normal([16, noise_dim])
    labels = tf.random.uniform([16], minval=0, maxval=num_classes, dtype=tf.int32)

    # Generate images
    generated_images = generator_model([noise, labels], training=False)

    # Convert generated images from [-1, 1] to [0, 1]
    generated_images = (generated_images + 1) / 2.0

    # Plot images and their labels
    fig, axes = plt.subplots(4, 4, figsize=(10, 10))
    axes = axes.flatten()

    for img, label, ax in zip(generated_images, labels, axes):
        ax.imshow(img.numpy().squeeze(), cmap='gray')
        ax.set_title(f'Label: {int(label)}')
        ax.axis('off')

    plt.tight_layout()
    return fig

In [51]:
#generate_images(generator, 100, 10)

In [53]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(60000).batch(256)